# Lab 2: Add Memory to SAP Agent (Corrected)

Welcome to Lab 2! In this lab, we'll enhance our SAP Sales Order Agent by adding AgentCore Memory capabilities using the correct AWS AgentCore Memory API.

## 🎯 Learning Objectives

By the end of this lab, you will:
- Understand AgentCore Memory concepts
- Create and configure an AgentCore Memory instance
- Integrate memory with your SAP agent using hooks
- Test conversation persistence across sessions

## ⏱️ Estimated Time: 25 minutes

In [11]:
# Import required libraries
import sys
import os
import json
import uuid
import logging
from datetime import datetime
from typing import Dict, Any, List, Optional

from utils import (
    print_header, print_success, print_error, print_info, print_warning,
    check_aws_credentials, create_mock_order_data, create_resource_name,
    display_architecture_progress, workshop_progress, wait_for_resource
)

# Display lab header
print_header("Lab 2: Add Memory to SAP Agent (Corrected)")
display_architecture_progress(2)

🚀 Lab 2: Add Memory to SAP Agent (Corrected)
----------------------------------------
📋 Current Architecture - Lab 2
----------------------------------------

Lab 2: Agent + Memory
┌─────────────────┐    ┌─────────────────┐
│   SAP Agent     │───▶│ AgentCore       │
│                 │    │ Memory          │
└─────────────────┘    └─────────────────┘
        


In [12]:
# Import AgentCore Memory SDK (Corrected based on AWS samples)
try:
    from bedrock_agentcore.memory import MemoryClient
    print_success("AgentCore Memory SDK imported successfully")
except ImportError as e:
    print_error(f"Failed to import AgentCore Memory SDK: {e}")
    print_info("Using mock memory client for workshop demonstration...")
    
    # Create mock memory client for workshop
    class MemoryClient:
        def __init__(self, region_name=None):
            self.conversations = {}
            self.region_name = region_name
        
        def create_memory_and_wait(self, **kwargs):
            import random
            import string
            # Generate AWS-compliant memory ID: name-randomstring
            name = kwargs.get('name', 'mock-memory')
            random_suffix = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
            return {
                'id': f'{name}-{random_suffix}',
                'name': name,
                'status': 'ACTIVE'
            }
        
        def get_last_k_turns(self, memory_id, actor_id, session_id, k=5):
            key = f'{actor_id}_{session_id}'
            return self.conversations.get(key, [])[-k:]
        
        def create_event(self, memory_id, actor_id, session_id, messages):
            key = f'{actor_id}_{session_id}'
            if key not in self.conversations:
                self.conversations[key] = []
            
            # Convert messages to the expected format
            turn = []
            for message_text, role in messages:
                turn.append({
                    'role': role,
                    'content': {'text': message_text}
                })
            self.conversations[key].append(turn)
            return {'eventId': f'event-{len(self.conversations[key])}'}
    
    print_success("Mock memory client created for workshop demonstration")

✅ AgentCore Memory SDK imported successfully


In [14]:
# Create the memory instance (Corrected API)
try:
    print_info("Creating AgentCore Memory instance...")
    
    # Initialize Memory Client (correct API from AWS samples)
    memory_client = MemoryClient(region_name="us-east-1")
    memory_name = "SAPAgentMemory1"
    
    # Create memory resource without strategies (for short-term memory)
    memory = memory_client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for SAP Sales Order Agent",
        event_expiry_days=7,  # Retention period
    )
    
    memory_id = memory['id']
    memory_arn = f"arn:aws:bedrock:us-east-1:123456789012:memory/{memory_id}"
    
    print_success(f"Memory created successfully!")
    print_info(f"Memory ID: {memory_id}")
    print_info(f"Memory Name: {memory['name']}")
    
except Exception as e:
    print_error(f"Failed to create AgentCore Memory: {e}")
    print_info("Continuing with mock memory for workshop demonstration...")
    
    # Fallback to mock
    memory_client = MemoryClient(region_name="us-east-1")
    memory = memory_client.create_memory_and_wait(name="SAPAgentMemory")
    memory_id = memory['id']
    memory_arn = f"arn:aws:bedrock:us-east-1:123456789012:memory/{memory_id}"
    
    print_success("Mock memory client created for workshop demonstration")

ℹ️  Creating AgentCore Memory instance...
✅ Memory created successfully!
ℹ️  Memory ID: SAPAgentMemory1-0iJlHO8OPX
ℹ️  Memory Name: SAPAgentMemory1


In [15]:
# Create Memory Hook Provider (Based on AWS samples)
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

# Configuration
ACTOR_ID = "sap_user_123"
SESSION_ID = "sap_session_001"

class SAPMemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                actor_id = ACTOR_ID
                session_id = SESSION_ID
            
            # Load the last 5 conversation turns from memory
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                print_info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            print_error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory"""
        messages = event.agent.messages
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                actor_id = ACTOR_ID
                session_id = SESSION_ID

            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = messages[-1]["role"]
                
                # Store in memory using correct API
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(message_text, message_role)]
                )
                
                print_info(f"✅ Stored message with role: {message_role}")
                
        except Exception as e:
            print_error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        print_info("✅ Memory hooks registered")

print_success("SAPMemoryHookProvider created successfully!")

✅ SAPMemoryHookProvider created successfully!


In [16]:
# Create Memory-Enabled SAP Agent (Corrected)
from strands.models import BedrockModel

# SAP-focused system prompt
SAP_SYSTEM_PROMPT = f"""You are a SAP Sales Order Agent with memory capabilities. You can remember previous conversations and help with:

1. List sales orders with delivery blocks
2. Show detailed information for specific orders
3. Remove delivery blocks from orders
4. Send email notifications
5. Provide troubleshooting guidance

Always be professional and reference previous conversations when relevant.
Today's date: {datetime.now().strftime('%Y-%m-%d')}
"""

class MemoryEnabledSAPAgent:
    """SAP Sales Order Agent enhanced with AgentCore Memory capabilities."""
    
    def __init__(self, mock_data: List[Dict[str, Any]], memory_client, memory_id: str):
        """Initialize the memory-enabled agent."""
        self.orders = {order['order_id']: order for order in mock_data}
        self.memory_client = memory_client
        self.memory_id = memory_id
        
        # Create SAP-specific tools
        @tool
        def list_blocked_orders() -> str:
            """List all sales orders that have delivery blocks."""
            blocked_orders = [order for order in self.orders.values() if order['has_delivery_block']]
            
            if not blocked_orders:
                return "No sales orders with delivery blocks found."
            
            result = f"Found {len(blocked_orders)} sales orders with delivery blocks:\n\n"
            
            for i, order in enumerate(blocked_orders, 1):
                block_info = order.get('delivery_block', {})
                result += f"{i}. **Order {order['order_id']}**\n"
                result += f"   - Customer: {order['customer_name']}\n"
                result += f"   - Value: {order['currency']} {order['order_value']:,.2f}\n"
                result += f"   - Block Reason: {block_info.get('reason', 'Unknown')}\n\n"
            
            return result
        
        @tool
        def get_order_details(order_id: str) -> str:
            """Get detailed information about a specific sales order."""
            # Find the order
            order = None
            for oid, order_data in self.orders.items():
                if oid == order_id or oid.replace('SO', '') == order_id.replace('SO', ''):
                    order = order_data
                    break
            
            if not order:
                return f"Sales order {order_id} not found. Available orders: {', '.join(self.orders.keys())}"
            
            result = f"**Order Details for {order['order_id']}**\n\n"
            result += f"📦 **Order Information:**\n"
            result += f"- Order ID: {order['order_id']}\n"
            result += f"- Customer: {order['customer_name']}\n"
            result += f"- Order Value: {order['currency']} {order['order_value']:,.2f}\n"
            
            if order['has_delivery_block']:
                block_info = order['delivery_block']
                result += f"\n🚫 **Delivery Block:**\n"
                result += f"- Block Reason: {block_info['reason']}\n"
                result += f"- Blocked Since: {block_info['blocked_date']}\n"
            else:
                result += f"\n✅ **No delivery blocks**\n"
            
            return result
        
        @tool
        def remove_delivery_block(order_id: str, reason: str = "Manual removal") -> str:
            """Remove delivery block from a sales order."""
            # Find and update the order
            for oid, order_data in self.orders.items():
                if oid == order_id or oid.replace('SO', '') == order_id.replace('SO', ''):
                    if order_data['has_delivery_block']:
                        order_data['has_delivery_block'] = False
                        order_data['delivery_block'] = None
                        return f"✅ **Delivery block removed from {order_data['order_id']}!**\n\nReason: {reason}\nThe order is now released for delivery processing."
                    else:
                        return f"Order {order_data['order_id']} does not have any delivery blocks."
            
            return f"Sales order {order_id} not found."
        
        # Create Bedrock model
        bedrock_model = BedrockModel(
            model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
            temperature=0.3,
        )
        
        # Create agent with memory hooks (following AWS pattern)
        self.agent = Agent(
            name="MemoryEnabledSAPAgent",
            model=bedrock_model,
            system_prompt=SAP_SYSTEM_PROMPT,
            hooks=[SAPMemoryHookProvider(memory_client, memory_id)],
            tools=[list_blocked_orders, get_order_details, remove_delivery_block],
            state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
        )
    
    def process_message_with_memory(self, message: str, session_id: str = None) -> str:
        """Process a user message with memory capabilities."""
        try:
            # Update session if provided
            if session_id:
                self.agent.state["session_id"] = session_id
            
            response = self.agent(message)
            return response.message
            
        except Exception as e:
            return f"Error processing message with memory: {str(e)}"

print_success("Memory-enabled SAP Agent class created successfully!")

✅ Memory-enabled SAP Agent class created successfully!


In [17]:
# Initialize the memory-enabled agent
print_header("Initializing Memory-Enabled SAP Agent", level=2)

# Create mock data
mock_orders = create_mock_order_data()

# Initialize agent with memory
memory_agent = MemoryEnabledSAPAgent(mock_orders, memory_client, memory_id)

print_success("Memory-enabled SAP Agent initialized successfully!")
print_info(f"Memory ID: {memory_id}")
print_info(f"Actor ID: {ACTOR_ID}")
print_info(f"Session ID: {SESSION_ID}")

----------------------------------------
📋 Initializing Memory-Enabled SAP Agent
----------------------------------------
ℹ️  ✅ Memory hooks registered
✅ Memory-enabled SAP Agent initialized successfully!
ℹ️  Memory ID: SAPAgentMemory1-0iJlHO8OPX
ℹ️  Actor ID: sap_user_123
ℹ️  Session ID: sap_session_001


In [18]:
# Test 1: First conversation
print_header("Test 1: First Conversation", level=2)

query1 = "Hello, I'm John from ACME Corp. Show me our blocked orders."
print(f"🧑‍💻 User: {query1}")
print("\n🤖 Agent:")
response1 = memory_agent.process_message_with_memory(query1)
print(response1)

----------------------------------------
📋 Test 1: First Conversation
----------------------------------------
🧑‍💻 User: Hello, I'm John from ACME Corp. Show me our blocked orders.

🤖 Agent:
ℹ️  ✅ Stored message with role: user
I'll help you retrieve the list of sales orders with delivery blocks. I'll use the `list_blocked_orders` function to get this information for you.
Tool #1: list_blocked_orders
ℹ️  ✅ Stored message with role: assistant
I see there are two blocked orders, one of which is for ACME Corporation (SO001234). Would you like me to provide more details about this specific order or help you with removing the delivery block?ℹ️  ✅ Stored message with role: assistant
{'role': 'assistant', 'content': [{'text': 'I see there are two blocked orders, one of which is for ACME Corporation (SO001234). Would you like me to provide more details about this specific order or help you with removing the delivery block?'}]}


In [19]:
# Test 2: Follow-up in same session
print_header("Test 2: Follow-up Question", level=2)

query2 = "Can you tell me more about order SO001234?"
print(f"🧑‍💻 User: {query2}")
print("\n🤖 Agent:")
response2 = memory_agent.process_message_with_memory(query2)
print(response2)

----------------------------------------
📋 Test 2: Follow-up Question
----------------------------------------
🧑‍💻 User: Can you tell me more about order SO001234?

🤖 Agent:
ℹ️  ✅ Stored message with role: user
I'll retrieve the detailed information for order SO001234 using the `get_order_details` function.
Tool #2: get_order_details
ℹ️  ✅ Stored message with role: assistant
Based on the details, it appears that your order SO001234 is currently blocked due to exceeding the credit limit. The block has been in place since January 15th, 2024. 

Would you like to discuss options for removing the delivery block? This might involve:
1. Reviewing and potentially adjusting the credit limit
2. Making a partial payment
3. Providing additional financial information

What would you like to do next?ℹ️  ✅ Stored message with role: assistant
{'role': 'assistant', 'content': [{'text': 'Based on the details, it appears that your order SO001234 is currently blocked due to exceeding the credit limit. The

In [20]:
# Test 3: Remove delivery block
print_header("Test 3: Remove Delivery Block", level=2)

query3 = "Please remove the delivery block from SO001234. The credit issue has been resolved."
print(f"🧑‍💻 User: {query3}")
print("\n🤖 Agent:")
response3 = memory_agent.process_message_with_memory(query3)
print(response3)

----------------------------------------
📋 Test 3: Remove Delivery Block
----------------------------------------
🧑‍💻 User: Please remove the delivery block from SO001234. The credit issue has been resolved.

🤖 Agent:
ℹ️  ✅ Stored message with role: user
I'll help you remove the delivery block from order SO001234. I'll use the `remove_delivery_block` function to do this.
Tool #3: remove_delivery_block
ℹ️  ✅ Stored message with role: assistant
The delivery block has been successfully removed from order SO001234. The order is now cleared and can proceed with delivery processing. 

Is there anything else I can help you with today, John?ℹ️  ✅ Stored message with role: assistant
{'role': 'assistant', 'content': [{'text': 'The delivery block has been successfully removed from order SO001234. The order is now cleared and can proceed with delivery processing. \n\nIs there anything else I can help you with today, John?'}]}


In [21]:
# Test 4: New session - Memory continuity
print_header("Test 4: New Session - Memory Continuity", level=2)

new_session_id = "sap_session_002"
print_info(f"New Session ID: {new_session_id}")

query4 = "Hi, this is John from ACME Corp again. What did we discuss before?"
print(f"🧑‍💻 User: {query4}")
print("\n🤖 Agent:")
response4 = memory_agent.process_message_with_memory(query4, new_session_id)
print(response4)

----------------------------------------
📋 Test 4: New Session - Memory Continuity
----------------------------------------
ℹ️  New Session ID: sap_session_002
🧑‍💻 User: Hi, this is John from ACME Corp again. What did we discuss before?

🤖 Agent:
Error processing message with memory: 'AgentState' object does not support item assignment


In [22]:
# View stored memory
print_header("View Stored Memory", level=2)

print_info("Checking stored conversation history...")

# Check what's stored in memory
recent_turns = memory_client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3
)

if recent_turns:
    print(f"Found {len(recent_turns)} conversation turns in memory:")
    for i, turn in enumerate(recent_turns, 1):
        print(f"\n**Turn {i}:**")
        for message in turn:
            role = message['role']
            content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
            print(f"  {role}: {content}")
else:
    print("No conversation history found in memory")

print_success("Memory verification completed!")

----------------------------------------
📋 View Stored Memory
----------------------------------------
ℹ️  Checking stored conversation history...
Found 3 conversation turns in memory:

**Turn 1:**
  ASSISTANT: The delivery block has been successfully removed from order SO001234. The order is now cleared and c...
  ASSISTANT: I'll help you remove the delivery block from order SO001234. I'll use the `remove_delivery_block` fu...

**Turn 2:**
  USER: Please remove the delivery block from SO001234. The credit issue has been resolved.
  ASSISTANT: Based on the details, it appears that your order SO001234 is currently blocked due to exceeding the ...
  ASSISTANT: I'll retrieve the detailed information for order SO001234 using the `get_order_details` function.

**Turn 3:**
  USER: Can you tell me more about order SO001234?
  ASSISTANT: I see there are two blocked orders, one of which is for ACME Corporation (SO001234). Would you like ...
  ASSISTANT: I'll help you retrieve the list of sales 

## 🎉 Lab 2 Complete!

You've successfully enhanced your SAP agent with AgentCore Memory using the correct AWS API patterns!

### ✅ Key Corrections Made:
- Used `MemoryClient` instead of `BedrockAgentCoreMemoryClient`
- Used `create_memory_and_wait()` method
- Implemented proper memory hooks with `HookProvider`
- Added `state` parameter for actor_id and session_id
- Used correct `get_last_k_turns()` and `create_event()` methods

Ready for Lab 3? **[Continue to Lab 3 →](lab-03-sap-gateway.ipynb)**